In [31]:
import os
import csv
import pandas as pd
import re
import time
import isodate
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from dotenv import load_dotenv

In [32]:
load_dotenv(override=True)  # Parámetro clave para sobrescribir si ya estaba cargada la api key

# La API KEY está expuesta en el archivo .env solo con fines académicos y será eliminada 14 días después de la entrega
api_key = os.getenv('YOUTUBE_API_KEY')

In [33]:
canales_interes = {
    'UCPH3Oz99Y_jrVBCQMjQZNSg': 'pro-ucraniano',       # Memorias de Pez
    'UC3tNpTOHsTnkmbX1M2sS4xg': 'pro-ucraniano',       # VisualPolitik
    'UCnsvJeZO4RigQ898WdDNoBw': 'noticiero',           # El País
    'UC7QZIf0dta-XPXsp9Hv4dTw': 'noticiero',           # RTVE Noticias
    'UClLLRs_mFTsNT5U-DqTYAGg': 'noticiero',           # La Vanguardia
    'UCwd8Byi93KbnsYmCcKLExvQ': 'pro-ruso',            # Negocios TV
    'UCgms7r9SaeYhuIBaPGOjnhw': 'pro-ruso',            # Miguel Ruiz Calvo
    'UCNKomgId0-uTA-vVLM9v1pw': 'pro-ruso',            # Intereconomía
    'UCGXbLrVe8vnkiFv7q2vYv3w': 'noticiero',           # El Mundo
    'UCCJs5mITIqxqJGeFjt9N1Mg': 'noticiero',           # laSexta Noticias
    'UCcgqSM4YEo5vVQpqwN-MaNw': 'pro-ruso',            # teleSUR
}


In [34]:
def buscar_videos_para_batch(canales, query='guerra ucrania', max_results=50):
    youtube = build('youtube', 'v3', developerKey=api_key)
    video_list = []

    for canal_id, clasificacion in canales.items():
        print(f"🔎 Buscando videos para canal {canal_id} ({clasificacion})...")

        request = youtube.search().list(
            q=query,
            channelId=canal_id,
            part='snippet',
            type='video',
            maxResults=max_results,
            publishedAfter='2024-01-01T00:00:00Z',
            publishedBefore='2024-12-31T23:59:59Z'
        )
        response = request.execute()

        for item in response.get('items', []):
            video_list.append({
                "video_id": item['id']['videoId'],
                "evento": "guerra ucrania",
                "tipo_evento": "geopolitico",
                "relacion_evento": "directa",
                "condiciones_cuenta": clasificacion
            })

        time.sleep(0.5)

    print(f"📊 Videos encontrados en total: {len(video_list)}")
    return video_list

In [ ]:
#  Cargar lista de videos ya procesados
def cargar_videos_procesados(path="comentarios_batch.csv"):
    if os.path.exists(path):
        df = pd.read_csv(path, usecols=['video_id'])
        return set(df['video_id'].unique())
    else:
        return set()

In [ ]:
# 🚀 Extracción incremental y segura
def extraer_comentarios_videos_incremental(video_list, api_key, output_path="comentarios_batch.csv"):
    procesados = cargar_videos_procesados(output_path)

    for video in video_list:
        if video['video_id'] in procesados:
            print(f"✅ Saltando (ya procesado): {video['video_id']}")
            continue

        print(f"📥 Procesando video: {video['video_id']} ({video['evento']})")
        comentarios = get_video_comments(
            video_id=video['video_id'],
            relacion_evento=video.get('relacion_evento', ''),
            evento=video.get('evento', ''),
            tipo_evento=video.get('tipo_evento', ''),
            condiciones_cuenta=video.get('condiciones_cuenta', ''),
            api_key=api_key
        )

        if comentarios:
            df_temp = pd.DataFrame(comentarios)
            if os.path.exists(output_path):
                df_temp.to_csv(output_path, mode='a', index=False, header=False)
            else:
                df_temp.to_csv(output_path, index=False)
            print(f"💾 Guardado {len(comentarios)} comentarios")
        time.sleep(1)

    print("🏁 Extracción finalizada.")

In [ ]:
def get_video_comments(video_id, relacion_evento, evento, tipo_evento, condiciones_cuenta, api_key, max_retries=5):
    youtube = build('youtube', 'v3', developerKey=api_key)

    try:
        video_response = youtube.videos().list(
            part='snippet,statistics,contentDetails',
            id=video_id
        ).execute()
        item = video_response['items'][0]
    except Exception as e:
        print(f"❌ Error al obtener info del video {video_id}: {e}")
        return []

    snippet = item['snippet']
    stats = item['statistics']
    content = item['contentDetails']

    video_data = {
        'video_title': snippet['title'],
        'channel_title': snippet['channelTitle'],
        'video_published_at': snippet['publishedAt'],
        'video_views': int(stats.get('viewCount', 0)),
        'video_likes': int(stats.get('likeCount', 0)),
        'video_duration': isodate.parse_duration(content['duration']).total_seconds(),
        'video_tags': snippet.get('tags', []),
        'video_category_id': snippet.get('categoryId', None),
        'relacion_evento': relacion_evento,
        'evento': evento,
        'tipo_evento': tipo_evento,
        'condiciones_cuenta': condiciones_cuenta
    }

    comentarios = []
    next_page_token = None
    retries = 0

    while True:
        try:
            response = youtube.commentThreads().list(
                part='snippet',
                videoId=video_id,
                maxResults=100,
                pageToken=next_page_token,
                textFormat='plainText'
            ).execute()

            for item in response.get('items', []):
                top_comment = item['snippet']['topLevelComment']['snippet']
                comentarios.append({
                    'comment_id': item['snippet']['topLevelComment']['id'],
                    'comment': top_comment['textDisplay'],
                    'comment_text_length': len(top_comment['textDisplay']),
                    'user_id': top_comment['authorChannelId']['value'],
                    'user_name': top_comment['authorDisplayName'],
                    'comment_time': top_comment['publishedAt'],
                    'comment_likes': top_comment['likeCount'],
                    'total_reply_count': item['snippet']['totalReplyCount'],
                    'is_top_level_comment': True,
                    **video_data
                })

            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break

        except HttpError as e:
            if e.resp.status in [500, 503]:
                retries += 1
                if retries > max_retries:
                    print(f"⚠️ Reintentos agotados para {video_id}")
                    break
                sleep_time = 2 ** retries
                print(f"⏳ Error {e.resp.status}, reintentando en {sleep_time} segundos...")
                time.sleep(sleep_time)
            else:
                print(f"❌ Error irrecuperable: {e}")
                break

    return comentarios

# viejo

In [ ]:
# Función para buscar videos en un canal sobre "guerra Ucrania" en 2024
def buscar_videos_canal(channel_id, query='guerra ucrania', published_after='2024-01-01T00:00:00Z', published_before='2024-12-31T23:59:59Z', max_results=10):
    youtube = build('youtube', 'v3', developerKey=api_key)
    request = youtube.search().list(
        q=query,
        channelId=channel_id,
        part='snippet',
        type='video',
        maxResults=max_results,
        publishedAfter=published_after,
        publishedBefore=published_before
    )
    response = request.execute()
    return response.get('items', [])

In [ ]:
# Función para obtener metadatos del video
def obtener_info_video(video_id):
    youtube = build('youtube', 'v3', developerKey=api_key)
    response = youtube.videos().list(
        part='snippet,statistics,contentDetails',
        id=video_id
    ).execute()
    if response['items']:
        item = response['items'][0]
        return {
            'video_id': video_id,
            'video_title': item['snippet']['title'],
            'channel_title': item['snippet']['channelTitle'],
            'video_published_at': item['snippet']['publishedAt'],
            'video_views': int(item['statistics'].get('viewCount', 0)),
            'video_likes': int(item['statistics'].get('likeCount', 0)),
            'video_duration': isodate.parse_duration(item['contentDetails']['duration']).total_seconds(),
            'video_tags': item['snippet'].get('tags', []),
            'video_category_id': item['snippet']['categoryId']
        }
    return None


In [ ]:
# 💬 Función para extraer comentarios
def obtener_comentarios(video_id, video_info, condiciones_cuenta, max_total=1000):
    youtube = build('youtube', 'v3', developerKey=api_key)
    comentarios = []
    next_page_token = None
    total_recibidos = 0

    while True:
        request = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=100,
            textFormat='plainText',
            pageToken=next_page_token
        )
        response = request.execute()

        for item in response.get('items', []):
            top_comment = item['snippet']['topLevelComment']['snippet']
            comentarios.append({
                'comment_id': item['snippet']['topLevelComment']['id'],
                'comment': top_comment['textDisplay'],
                'comment_text_length': len(top_comment['textDisplay']),
                'user_id': top_comment['authorChannelId']['value'],
                'user_name': top_comment['authorDisplayName'],
                'comment_time': top_comment['publishedAt'],
                'comment_likes': top_comment['likeCount'],
                'total_reply_count': item['snippet']['totalReplyCount'],
                'is_top_level_comment': True,
                'video_title': video_info['video_title'],
                'channel_title': video_info['channel_title'],
                'video_published_at': video_info['video_published_at'],
                'video_views': video_info['video_views'],
                'video_likes': video_info['video_likes'],
                'video_duration': video_info['video_duration'],
                'video_tags': video_info['video_tags'],
                'video_category_id': video_info['video_category_id'],
                'condiciones_cuenta': condiciones_cuenta
            })
            total_recibidos += 1
            if total_recibidos >= max_total:
                break

        if 'nextPageToken' in response and total_recibidos < max_total:
            next_page_token = response['nextPageToken']
            time.sleep(0.5)
        else:
            break

    return comentarios



In [ ]:
canales_interes = {
    'UCPH3Oz99Y_jrVBCQMjQZNSg': 'pro-ucraniano',       # Memorias de Pez
    'UC3tNpTOHsTnkmbX1M2sS4xg': 'pro-ucraniano',       # VisualPolitik
    'UCnsvJeZO4RigQ898WdDNoBw': 'noticiero',           # El País
    'UC7QZIf0dta-XPXsp9Hv4dTw': 'noticiero',           # RTVE Noticias
    'UClLLRs_mFTsNT5U-DqTYAGg': 'noticiero',           # La Vanguardia
    'UCwd8Byi93KbnsYmCcKLExvQ': 'pro-ruso',            # Negocios TV
    'UCgms7r9SaeYhuIBaPGOjnhw': 'pro-ruso',            # Miguel Ruiz Calvo
    'UCNKomgId0-uTA-vVLM9v1pw': 'pro-ruso',            # Intereconomía
    'UCGXbLrVe8vnkiFv7q2vYv3w': 'noticiero',           # El Mundo
    'UCCJs5mITIqxqJGeFjt9N1Mg': 'noticiero',           # laSexta Noticias
    'UCcgqSM4YEo5vVQpqwN-MaNw': 'pro-ruso',            # teleSUR
}


In [ ]:
def buscar_videos_relevantes_todos(canales, query='guerra ucrania'):
    todos_los_videos = []
    for canal_id, clasificacion in canales.items():
        print(f"🔎 Buscando videos para canal {canal_id} ({clasificacion})...")
        videos = buscar_videos_canal(canal_id, query=query)
        for v in videos:
            todos_los_videos.append({
                'video_id': v['id']['videoId'],
                'channel_id': canal_id,
                'clasificacion': clasificacion
            })
        time.sleep(0.5)
    return todos_los_videos


In [ ]:
def extraer_comentarios_de_todos(videos_info, max_comentarios=500):
    all_comments = []
    for video in videos_info:
        video_id = video['video_id']
        condiciones_cuenta = video['clasificacion']
        video_info = obtener_info_video(video_id)
        if video_info:
            print(f"📥 Extrayendo comentarios de: {video_info['video_title']}")
            comentarios = obtener_comentarios(video_id, video_info, condiciones_cuenta, max_total=max_comentarios)
            all_comments.extend(comentarios)
            time.sleep(1)
    return pd.DataFrame(all_comments)


In [ ]:
# Paso 1: buscar los videos de todos los canales
videos_filtrados = buscar_videos_relevantes_todos(canales_interes)

In [ ]:
# Paso 2: traer todos los comentarios de esos videos
df_comentarios_todos = extraer_comentarios_de_todos(videos_filtrados, max_comentarios=200)

In [25]:
# ▶️ Ejemplo real: buscar y extraer comentarios del primer video encontrado de un canal

# Canal: Memorias de Pez (pro-ucraniano)
channel_id = 'UCPH3Oz99Y_jrVBCQMjQZNSg'
condiciones_cuenta = 'pro-ucraniano'

# Buscar videos relacionados a "guerra ucrania" en 2024
videos = buscar_videos_canal(channel_id, query='guerra ucrania', max_results=1)

if videos:
    video_id = videos[0]['id']['videoId']
    video_info = obtener_info_video(video_id)
    comentarios = obtener_comentarios(video_id, video_info, condiciones_cuenta)

    # Convertir a DataFrame
    df_comentarios = pd.DataFrame(comentarios)
else:
    print("No se encontraron videos sobre 'guerra ucrania' en este canal para 2024.")


In [29]:
df_comentarios.head()

,comment_id,comment,user_id,user_name,comment_time,comment_likes,total_reply_count,is_top_level_comment,video_title,channel_title,video_published_at,video_views,video_likes,video_duration,condiciones_cuenta
0,Ugy7XNqU8Smzq3GfBVF4AaABAg,Este se nota que cuando Rusia consigue algo en...,UCocb_0i4-sfZbAuA9oGXeUQ,@herrgottsir,2025-04-14T22:54:01Z,0,0,True,✅ RESUMEN de la SEMANA 142 de guerra entre UCR...,Memorias de Pez,2024-11-21T17:37:22Z,210027,13362,307.0,pro-ucraniano
1,UgyZjQOfHF_EkutLxeR4AaABAg,"0:07 cuando terminara esta locura,,,ya Rusia g...",UCIOGqwO1MIR0hZ0d4Kyx73w,@davidtomasjaenbarsa172,2025-04-07T14:30:04Z,0,0,True,✅ RESUMEN de la SEMANA 142 de guerra entre UCR...,Memorias de Pez,2024-11-21T17:37:22Z,210027,13362,307.0,pro-ucraniano
2,UgwnBt4AkQC2dLUvzXx4AaABAg,"Hola como están, fíjense que el título está eq...",UCpVHHwi3SvCE19wxVlq7y6w,@gcr8844,2025-01-08T12:29:06Z,0,0,True,✅ RESUMEN de la SEMANA 142 de guerra entre UCR...,Memorias de Pez,2024-11-21T17:37:22Z,210027,13362,307.0,pro-ucraniano
3,UgwYCwjSliYyUwjVKOp4AaABAg,Zzzzzz,UCTTwT8e9Q_KiFh3qf--YoKg,@xforce2001,2024-12-20T16:57:22Z,0,0,True,✅ RESUMEN de la SEMANA 142 de guerra entre UCR...,Memorias de Pez,2024-11-21T17:37:22Z,210027,13362,307.0,pro-ucraniano
4,Ugyrvze7qQFTxB4gv0d4AaABAg,Vamos Rusia 😊,UCGjcyIO91D7-rLxigikXUyg,@Gaby_Gamer22,2024-12-08T20:01:22Z,1,0,True,✅ RESUMEN de la SEMANA 142 de guerra entre UCR...,Memorias de Pez,2024-11-21T17:37:22Z,210027,13362,307.0,pro-ucraniano


In [26]:
df_comentarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316 entries, 0 to 315
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   comment_id            316 non-null    object 
 1   comment               316 non-null    object 
 2   user_id               316 non-null    object 
 3   user_name             316 non-null    object 
 4   comment_time          316 non-null    object 
 5   comment_likes         316 non-null    int64  
 6   total_reply_count     316 non-null    int64  
 7   is_top_level_comment  316 non-null    bool   
 8   video_title           316 non-null    object 
 9   channel_title         316 non-null    object 
 10  video_published_at    316 non-null    object 
 11  video_views           316 non-null    int64  
 12  video_likes           316 non-null    int64  
 13  video_duration        316 non-null    float64
 14  condiciones_cuenta    316 non-null    object 
dtypes: bool(1), float64(1),

In [27]:
df_comentarios.to_csv('../data/processed/comentarios_completos.csv', index=False)
